In [ ]:
%pip install ollama langchain-ollama langchain-core

In [5]:
%pip install -qU langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
%pip install -qU langchain[serve]

Note: you may need to restart the kernel to use updated packages.


In [2]:
from fastapi import FastAPI
from langchain.prompts import ChatPromptTemplate
from langserve import add_routes
import uvicorn
import os

In [ ]:
llm = OllamaLLM(model="qwen2.5:0.5b")

RAG_PROMPT_TEMPLATE = """\
<|start_header_id|>system<|end_header_id|>
You are a helpful assistant. You answer user questions based on provided context. If you can't answer the question with the provided context, say you don't know.<|eot_id|>

<|start_header_id|>user<|end_header_id|>
User Query:
{query}

Context:
{context}<|eot_id|>

<|start_header_id|>assistant<|end_header_id|>
"""

rag_prompt = PromptTemplate.from_template(RAG_PROMPT_TEMPLATE)


In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="all-minilm:latest",
)

In [ ]:
from langchain_community.vectorstores import Qdrant


qdrant_vectorstore = Qdrant.from_existing_collection(
    embedding=embeddings,
    url=os.environ['QDRANT_URL'],
    path=None,
    collection_name="eb1a_visa",    
    api_key=os.environ['QDRANT_API_KEY'],
)

retriever = qdrant_vectorstore.as_retriever()

In [ ]:


qdrant_vectorstore = QdrantVectorStore.from_existing_collection(
    embedding=embeddings,
    collection_name="DnD_Documents",
    prefer_grpc=True,
    url=url
)



lcel_rag_chain = {"context": itemgetter("query") | retriever, "query": itemgetter("query")}| rag_prompt | llm

In [3]:
app = FastAPI(
    title="Langchain Server",
    version="1.0",
    description="A simple API server"
)

add_routes(
    app,
    
)

In [4]:
import ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="qwen2.5:0.5b")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})